In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
# 버튼 클릭을 위한 함수 정의
def click_button_by_class_name(class_name):
    try:
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, class_name)))
        button.click()
    except Exception as e:
        print(f"버튼 클릭 실패: {e}")

In [3]:
def click_button_by_data_value(data_value):
    try:
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'[data-value="{data_value}"]')))
        button.click()
    except Exception as e:
        print(f"버튼 클릭 실패: {e}")

In [133]:
def page_click(num):
    if num >= 11:
        num = num-9 # 12번째 페이지(num=11)의 XPATH : //*[@id="toplistWrapper"]/div[2]/div/a[2]
    
    _path = '//*[@id="toplistWrapper"]/div[2]/div/a['+str(num)+']'
    driver.find_element_by_xpath(_path).click() # 다음 페이지 클릭
    
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    link_element= soup.select('a.pcol2._setTop._setTopListUrl')
    
    logNo_tmp = []
    links_tmp = []

    for link in link_element:
        link = str(link)
        start_pos = link.find('logNo=')+6
        end_pos = link.find('&amp;',start_pos)
        value = link[start_pos:end_pos]
        logNo_tmp.append(value)
    
    for num in logNo_tmp:
        txt = "https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo="+str(num)
        links_tmp.append(txt)

    time.sleep(2)
    
    return logNo_tmp, links_tmp

# 1. url 추출(KB부동산 기준)

In [57]:
# 웹 드라이버 실행
driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')

# 해당 페이지로 이동
url = 'https://blog.naver.com/PostList.naver?blogId=youngkbblog&categoryNo=18&from=postList' 
driver.get(url)

# 버튼 클릭 시뮬레이션
click_button_by_class_name('btn_openlist')
print('목록열기 버튼')

click_button_by_class_name('btn_select')
print('보기 버튼')

click_button_by_data_value("30")
print('30줄 보기 버튼')

time.sleep(2)

## 첫번째 페이지
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print('======================= 1 번째 페이지 =====================')
link_element= soup.select('a.pcol2._setTop._setTopListUrl')

logNo_list = []
links_list = []
for link in link_element:
    link = str(link)
    start_pos = link.find('logNo=')+6
    end_pos = link.find('&amp;',start_pos)
    value = link[start_pos:end_pos]
    logNo_list.append(value)
    
for num in logNo_list:
    txt = "https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo="+str(num)
    links_list.append(txt)
    
print('게시물 개수 : ',len(logNo_list))
print(links_list[0])

time.sleep(2)

print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수

page_cnt = 6 # 30줄씩 했을 때 페이지 개수

for i in range(1,page_cnt):
    print('=======================',i+1,'번째 페이지 =====================')
    no, link = page_click(i)
    print('게시물 개수 : ',len(no))
    print(link[0])
    logNo_list.extend(no)
    links_list.extend(link)
    print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수
    time.sleep(1)

목록열기 버튼
보기 버튼
30줄 보기 버튼
======================= 1 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=222999006126
0
======================= 2 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221808507890
0
======================= 3 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221638251271
0
======================= 4 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221556306103
0
======================= 5 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221510853801
0
======================= 6 번째 페이지 =====================
게시물 개수 :  8
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221460663620
0


## 중복확인

In [58]:
import pandas as pd
pd.DataFrame(logNo_list)

,0
0,222999006126
1,222966323795
2,222935215965
3,222885402631
4,222857948328
...,...
153,221456374977
154,221455562333
155,221455014983
156,221454898363


In [59]:
print(pd.DataFrame(logNo_list).duplicated(keep = False).sum())

0


In [60]:
pd.DataFrame(logNo_list)[pd.DataFrame(logNo_list).duplicated(keep = False)]

,0


# 2. 정보 추출

In [112]:
import requests
from bs4 import BeautifulSoup
import re
    
def get_blog_post_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        content_element = soup.select_one('.se-main-container')
        title = soup.find("meta",property="og:title")['content']
        date_element = soup.select_one('.se_publishDate.pcol2')
        category_element = soup.select_one('.blog2_series a.pcol2')

        if content_element and title and date_element and category_element:
            # 본문 내용의 텍스트를 추출하고, 여러 개의 공백을 하나로 줄임
            text = content_element.get_text(separator=' ')
            text_without_newlines_tabs_spaces = re.sub(r'\s+', ' ', text)
            date = date_element.get_text()
            category = category_element.get_text()
            return text_without_newlines_tabs_spaces.strip(), title, date, category
        else:
            print('====================================')
            if content_element is None:
                print('content 없음')
                text = 0
            else:
                text = content_element.get_text(separator=' ')
                text_without_newlines_tabs_spaces = re.sub(r'\s+', ' ', text)
                text = text_without_newlines_tabs_spaces
                
            if title is None:
                print('title 없음')
                title = 0
            
            if date_element is None:
                print('date 없음')
                date = 0
            else:
                date = date_element.get_text()
                
            if category_element is None:
                print('category 없음')
                category = 0
            else:
                category = category_element.get_text()
                
            return text, title, date, category

    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

In [104]:
le = None
if le is None:
    print('he')

he


In [68]:
content_list = []
title_list = []
date_list = []
category_list = []

cnt = 0
for tmp in links_list:
    print("============",cnt+1,'번째===========')
    cnt=cnt+1
    content,title,date,category = get_blog_post_text(tmp)
    content_list.append(content)
    title_list.append(title)
    date_list.append(date)
    category_list.append(category)

# 3. 데이터프레임 생성

In [72]:
blog_data = pd.DataFrame()

In [73]:
blog_data['category'] = category_list
blog_data['title'] = title_list
blog_data['content'] = content_list
blog_data['date'] = date_list
blog_data['logNo'] = logNo_list
blog_data['link'] = links_list

In [74]:
blog_data.head()

,category,title,content,date,logNo,link
0,KB부동산,무주택자가 불안한 하락장에서 내 집 마련 성공하려면? (Feat. 부동산 생애 주기),"요즘 부동산 시장을 보면, 무주택자 여러분들의 고민이 깊어질 것 같습니다. 집을 사...",2023. 1. 30. 14:27,222999006126,https://blog.naver.com/PostView.naver?blogId=y...
1,KB부동산,초고가 아파트의 강세가 이어지는 까닭은?,금리 인상으로 인한 부동산 한파가 꽤나 오래 지속되고 있습니다. 올해는 모든 부동산...,2022. 12. 27. 10:00,222966323795,https://blog.naver.com/PostView.naver?blogId=y...
2,KB부동산,월세 낮아서 계약했더니 관리비가 월세 2배? 관리비 꼼수 주의보,A씨는 올해 원룸 월세 계약을 알아보다 파격적인 월세 매물을 찾았습니다. 보증금 3...,2022. 11. 22. 13:25,222935215965,https://blog.naver.com/PostView.naver?blogId=y...
3,KB부동산,6년만 살면 내 집이 된다? '내 집 마련 리츠' 바로 알기,향후 5년간 270만호 주택 공급이 계획이 담긴 8.16 부동산대책에는 리츠 주택 ...,2022. 9. 27. 11:21,222885402631,https://blog.naver.com/PostView.naver?blogId=y...
4,KB부동산,무주택자라면 집중해야 할 올 하반기 사전청약 완벽 정리!,​ ​ 7월을 시작으로 올해 하반기 사전청약이 시작됐습니다. 특히 많은 관심을 모으...,2022. 8. 26. 15:00,222857948328,https://blog.naver.com/PostView.naver?blogId=y...


In [75]:
blog_data.to_csv("blog_real_estate.csv", index = False)

# 4. 다른 카테고리도 진행  
- kb뉴스, kb상품, kb라이프

## KB뉴스 - 예외 존재

In [110]:
# 웹 드라이버 실행
driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')

# 해당 페이지로 이동
##================================== 수정 필요 =========================================##
url = 'https://blog.naver.com/PostList.nhn?blogId=youngkbblog&from=postList&categoryNo=13' 
##========================================================================================##
driver.get(url)

# 버튼 클릭 시뮬레이션
click_button_by_class_name('btn_openlist')
print('목록열기 버튼')

click_button_by_class_name('btn_select')
print('보기 버튼')

click_button_by_data_value("30")
print('30줄 보기 버튼')

time.sleep(2)

## 첫번째 페이지
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print('======================= 1 번째 페이지 =====================')
link_element= soup.select('a.pcol2._setTop._setTopListUrl')

logNo_list = []
links_list = []
for link in link_element:
    link = str(link)
    start_pos = link.find('logNo=')+6
    end_pos = link.find('&amp;',start_pos)
    value = link[start_pos:end_pos]
    logNo_list.append(value)
    
for num in logNo_list:
    txt = "https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo="+str(num)
    links_list.append(txt)
    
print('게시물 개수 : ',len(logNo_list))
print(links_list[0])

time.sleep(2)

print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수

##============= 수정 ====================##
page_cnt = 4 # 30줄씩 했을 때 페이지 개수
##=======================================##

for i in range(1,page_cnt):
    print('=======================',i+1,'번째 페이지 =====================')
    no, link = page_click(i)
    print('게시물 개수 : ',len(no))
    print(link[0])
    logNo_list.extend(no)
    links_list.extend(link)
    print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수
    time.sleep(1)

목록열기 버튼
보기 버튼
30줄 보기 버튼
======================= 1 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=222680930411
0
======================= 2 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221994138449
0
======================= 3 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221509418479
0
======================= 4 번째 페이지 =====================
게시물 개수 :  25
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221432447263
0


In [113]:
content_list = []
title_list = []
date_list = []
category_list = []
except_link = [] ## 예외

cnt = 0
for tmp in links_list:
    print("============",cnt+1,'번째===========')
    cnt=cnt+1
    
    try:
        content,title,date,category = get_blog_post_text(tmp)
        content_list.append(content)
        title_list.append(title)
        date_list.append(date)
        category_list.append(category)
        
    except: # 그 전에 이미 오류 처리
        print('오류 발생')
        except_link.append(tmp)
        content_list.append('error')
        title_list.append('error')
        date_list.append('error')
        category_list.append('error')

============ 1 번째===========
============ 2 번째===========
============ 3 번째===========
============ 4 번째===========
============ 5 번째===========
============ 6 번째===========
============ 7 번째===========
============ 8 번째===========
============ 9 번째===========
============ 10 번째===========
============ 11 번째===========
============ 12 번째===========
============ 13 번째===========
============ 14 번째===========
============ 15 번째===========
============ 16 번째===========
============ 17 번째===========
============ 18 번째===========
============ 19 번째===========
============ 20 번째===========
============ 21 번째===========
============ 22 번째===========
============ 23 번째===========
============ 24 번째===========
============ 25 번째===========
============ 26 번째===========
============ 27 번째===========
============ 28 번째===========
============ 29 번째===========
============ 30 번째===========
============ 31 번째===========
============ 32 번째===========
============ 33 번째===========
============ 34 번째=

In [118]:
blog_data = pd.DataFrame()

In [119]:
blog_data['category'] = category_list
blog_data['title'] = title_list
blog_data['content'] = content_list
blog_data['date'] = date_list
blog_data['logNo'] = logNo_list
blog_data['link'] = links_list

In [123]:
blog_data[94:]

,category,title,content,date,logNo,link
94,KB뉴스,KB국민은행 X 방탄소년단 PLAY DIGITAL KB 광고 티저 영상 공개!,0,2018. 12. 15. 0:00,221419377351,https://blog.naver.com/PostView.naver?blogId=y...
95,KB뉴스,KB국민은행 X 방탄소년단의 두 번째 광고! 티저 영상 리브(Liiv) 최초 공개!,0,2018. 12. 13. 17:18,221418517004,https://blog.naver.com/PostView.naver?blogId=y...
96,KB뉴스,국가고객만족도(NCSI) 은행권 최초 총 12회 1위 선정!,0,2018. 12. 6. 11:21,221413523959,https://blog.naver.com/PostView.naver?blogId=y...
97,KB뉴스,성공 취업을 KB가 응원합니다!,0,2018. 11. 23. 11:20,221404467089,https://blog.naver.com/PostView.naver?blogId=y...
98,KB뉴스,KB국민은행의 17번째 생일을 축하합니다!,0,2018. 11. 1. 15:01,221389501647,https://blog.naver.com/PostView.naver?blogId=y...
99,KB뉴스,방탄소년단과 함께 특별한 가상체험! KB★Stargram!,0,2018. 9. 27. 15:13,221366362622,https://blog.naver.com/PostView.naver?blogId=y...
100,KB뉴스,"KB블로그지기,GOLD&WISE 콘서트에 가다",0,2018. 9. 13. 15:08,221358223268,https://blog.naver.com/PostView.naver?blogId=y...
101,KB뉴스,KB블로그지기의 2018 FMS SEOUL 방문기,0,2018. 9. 11. 16:24,221356860525,https://blog.naver.com/PostView.naver?blogId=y...
102,KB뉴스,"KB블로그지기, KB국민은행 채용담당자를 만나다",0,2018. 9. 7. 14:03,221354318855,https://blog.naver.com/PostView.naver?blogId=y...
103,KB뉴스,블로그지기의 KB 스타그램 방문기,0,2018. 9. 6. 15:20,221353659151,https://blog.naver.com/PostView.naver?blogId=y...


In [124]:
blog_data.to_csv("blog_news.csv", index = False)

## KB상품 - 예외 존재

In [125]:
# 웹 드라이버 실행
driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')

# 해당 페이지로 이동
##=================================== 수정 필요 ===========================================##
url = 'https://blog.naver.com/PostList.nhn?blogId=youngkbblog&from=postList&categoryNo=14' 
##========================================================================================##
driver.get(url)

# 버튼 클릭 시뮬레이션
click_button_by_class_name('btn_openlist')
print('목록열기 버튼')

click_button_by_class_name('btn_select')
print('보기 버튼')

click_button_by_data_value("30")
print('30줄 보기 버튼')

time.sleep(2)

## 첫번째 페이지
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print('======================= 1 번째 페이지 =====================')
link_element= soup.select('a.pcol2._setTop._setTopListUrl')

logNo_list = []
links_list = []
for link in link_element:
    link = str(link)
    start_pos = link.find('logNo=')+6
    end_pos = link.find('&amp;',start_pos)
    value = link[start_pos:end_pos]
    logNo_list.append(value)
    
for num in logNo_list:
    txt = "https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo="+str(num)
    links_list.append(txt)
    
print('게시물 개수 : ',len(logNo_list))
print(links_list[0])

time.sleep(2)

print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수

##============= 수정 ====================##
page_cnt = 1 # 30줄씩 했을 때 페이지 개수
##=======================================##

for i in range(1,page_cnt):
    print('=======================',i+1,'번째 페이지 =====================')
    no, link = page_click(i)
    print('게시물 개수 : ',len(no))
    print(link[0])
    logNo_list.extend(no)
    links_list.extend(link)
    print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수
    time.sleep(1)

목록열기 버튼
보기 버튼
30줄 보기 버튼
======================= 1 번째 페이지 =====================
게시물 개수 :  1
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=221409300930
0


In [126]:
content_list = []
title_list = []
date_list = []
category_list = []
except_link2 = [] ## 예외

cnt = 0
for tmp in links_list:
    print("============",cnt+1,'번째===========')
    cnt=cnt+1
    
    try:
        content,title,date,category = get_blog_post_text(tmp)
        content_list.append(content)
        title_list.append(title)
        date_list.append(date)
        category_list.append(category)
        
    except:
        print('오류 발생')
        except_link2.append(tmp)
        content_list.append('error')
        title_list.append('error')
        date_list.append('error')
        category_list.append('error')

print(except_link2)

============ 1 번째===========
content 없음
[]


In [127]:
blog_data = pd.DataFrame()

In [128]:
blog_data['category'] = category_list
blog_data['title'] = title_list
blog_data['content'] = content_list
blog_data['date'] = date_list
blog_data['logNo'] = logNo_list
blog_data['link'] = links_list

In [129]:
blog_data.head()

,category,title,content,date,logNo,link
0,KB상품,퇴근하고도 은행 방문이 가능하다? 7시까지 문여는 KB국민은행 영업시간 특화 점포,0,2018. 11. 30. 10:28,221409300930,https://blog.naver.com/PostView.naver?blogId=y...


In [130]:
blog_data.to_csv("blog_products.csv", index = False)

## KB라이프 - 예외 존재

In [134]:
# 웹 드라이버 실행
driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')

# 해당 페이지로 이동
##================================== 수정 필요 ===========================================##
url = 'https://blog.naver.com/PostList.nhn?blogId=youngkbblog&from=postList&categoryNo=15' 
##========================================================================================##
driver.get(url)

# 버튼 클릭 시뮬레이션
click_button_by_class_name('btn_openlist')
print('목록열기 버튼')

click_button_by_class_name('btn_select')
print('보기 버튼')

click_button_by_data_value("30")
print('30줄 보기 버튼')

time.sleep(2)

## 첫번째 페이지
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print('======================= 1 번째 페이지 =====================')
link_element= soup.select('a.pcol2._setTop._setTopListUrl')

logNo_list = []
links_list = []
for link in link_element:
    link = str(link)
    start_pos = link.find('logNo=')+6
    end_pos = link.find('&amp;',start_pos)
    value = link[start_pos:end_pos]
    logNo_list.append(value)
    
for num in logNo_list:
    txt = "https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo="+str(num)
    links_list.append(txt)
    
print('게시물 개수 : ',len(logNo_list))
print(links_list[0])

time.sleep(2)

print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수

##============= 수정 ====================##
page_cnt = 12 # 30줄씩 했을 때 페이지 개수
##=======================================##

for i in range(1,page_cnt):
    print('=======================',i+1,'번째 페이지 =====================')
    no, link = page_click(i)
    print('게시물 개수 : ',len(no))
    print(link[0])
    logNo_list.extend(no)
    links_list.extend(link)
    print(pd.DataFrame(logNo_list).duplicated(keep = False).sum()) ## 중복 개수
    time.sleep(1)

목록열기 버튼
보기 버튼
30줄 보기 버튼
======================= 1 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=223168425712
0
======================= 2 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=222962797549
0
======================= 3 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=222735878991
0
======================= 4 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=222515171988
0
======================= 5 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=222254598029
0
======================= 6 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.naver?blogId=youngkbblog&logNo=222117545385
0
======================= 7 번째 페이지 =====================
게시물 개수 :  30
https://blog.naver.com/PostView.

In [135]:
content_list = []
title_list = []
date_list = []
category_list = []
except_link3 = [] ## 예외

cnt = 0
for tmp in links_list:
    print("============",cnt+1,'번째===========')
    cnt=cnt+1
    
    try:
        content,title,date,category = get_blog_post_text(tmp)
        content_list.append(content)
        title_list.append(title)
        date_list.append(date)
        category_list.append(category)
        
    except:
        print('오류 발생')
        except_link3.append(tmp)
        content_list.append('error')
        title_list.append('error')
        date_list.append('error')
        category_list.append('error')

print(except_link3)

============ 1 번째===========
============ 2 번째===========
============ 3 번째===========
============ 4 번째===========
============ 5 번째===========
============ 6 번째===========
============ 7 번째===========
============ 8 번째===========
============ 9 번째===========
============ 10 번째===========
============ 11 번째===========
============ 12 번째===========
============ 13 번째===========
============ 14 번째===========
============ 15 번째===========
============ 16 번째===========
============ 17 번째===========
============ 18 번째===========
============ 19 번째===========
============ 20 번째===========
============ 21 번째===========
============ 22 번째===========
============ 23 번째===========
============ 24 번째===========
============ 25 번째===========
============ 26 번째===========
============ 27 번째===========
============ 28 번째===========
============ 29 번째===========
============ 30 번째===========
============ 31 번째===========
============ 32 번째===========
============ 33 번째===========
============ 34 번째=

============ 269 번째===========
============ 270 번째===========
============ 271 번째===========
============ 272 번째===========
============ 273 번째===========
============ 274 번째===========
============ 275 번째===========
============ 276 번째===========
============ 277 번째===========
============ 278 번째===========
============ 279 번째===========
============ 280 번째===========
============ 281 번째===========
============ 282 번째===========
============ 283 번째===========
============ 284 번째===========
============ 285 번째===========
============ 286 번째===========
============ 287 번째===========
============ 288 번째===========
============ 289 번째===========
============ 290 번째===========
============ 291 번째===========
============ 292 번째===========
============ 293 번째===========
============ 294 번째===========
============ 295 번째===========
============ 296 번째===========
============ 297 번째===========
============ 298 번째===========
============ 299 번째===========
============ 300 번째===========
========

In [136]:
blog_data = pd.DataFrame()

In [137]:
blog_data['category'] = category_list
blog_data['title'] = title_list
blog_data['content'] = content_list
blog_data['date'] = date_list
blog_data['logNo'] = logNo_list
blog_data['link'] = links_list

In [139]:
blog_data

,category,title,content,date,logNo,link
0,KB라이프,"[KB경영연구소] 애플, 은행이 되고자 하는가?",​ 애플세이빙스의 출시 ​ 애플은 지난 4월 말 골드만삭스와 협력하여 애플세이빙스 ...,2023. 7. 28. 15:00,223168425712,https://blog.naver.com/PostView.naver?blogId=y...
1,KB라이프,도시 한가운데 양봉장이? 사라져가는 꿀벌을 지켜주세요!,"꿀벌이 자취를 감추고 있다는 소식, 들어보셨나요? ​ 2020년 4월부터 2021년...",2023. 7. 21. 11:00,223161279034,https://blog.naver.com/PostView.naver?blogId=y...
2,KB라이프,지구에도 끝이 있다?! 지구 생태용량 초과의 날,​ 여러분 혹시 지구 생태용량 초과의 날 (Earth Overshoot Day)에 ...,2023. 7. 14. 11:41,223155900654,https://blog.naver.com/PostView.naver?blogId=y...
3,KB라이프,"해외여행 가기 전, KB스타뱅킹에서 나에게 꼭 맞는 환전하세요!",안녕하세요 여러분 최근 코로나19가 줄어들고 휴가철이 다가오면서 다시금 해외로 여행...,2023. 7. 7. 14:40,223149533864,https://blog.naver.com/PostView.naver?blogId=y...
4,KB라이프,"발전기 없는 발전소, 가상발전소(VPP)","​ 발전기 없는 발전소, 가상발전소(VPP) ​ 가상발전소(VPP, Virtual ...",2023. 6. 23. 11:00,223135978051,https://blog.naver.com/PostView.naver?blogId=y...
...,...,...,...,...,...,...
344,KB라이프,"통학러, 통근러 주목~! 지하철 비용 아끼는 꿀이득 지하철 정기권",0,2018. 8. 9. 14:00,221335500728,https://blog.naver.com/PostView.naver?blogId=y...
345,KB라이프,자취방 구할 때 이것만은 꼭! 자취방 점검 체크리스트 팁!,0,2018. 8. 1. 10:30,221330503092,https://blog.naver.com/PostView.naver?blogId=y...
346,KB라이프,삼복더위에는 잘 먹고 힘내기! 더위 싹 날아가는 이열치열 복날 보양식!,0,2018. 7. 27. 10:26,221327560643,https://blog.naver.com/PostView.naver?blogId=y...
347,KB라이프,"첫 직장, 첫 월급! 사회 초년생들을 위한 돈 관리 꿀팁!",0,2018. 7. 18. 12:08,221321384305,https://blog.naver.com/PostView.naver?blogId=y...


In [140]:
blog_data.to_csv("blog_life.csv", index = False)